# Summary Chapter 6 - Multilabel classification

Build a network that not only identifies whether there is target present but also activates if there are more than one target in the image or tells us if there is none. 

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

#hide
from fastbook import *

Fetch data and take a look at the included CSV file using Panda dataframes:

In [1]:
from fastai.vision.all import *
path = untar_data(URLs.PASCAL_2007)

df = pd.read_csv(path/'train.csv')
df.head()

ModuleNotFoundError: No module named 'fastai'

The csv file includes the following information: 
filename, objects included in the image and whether that image is part of the training or validation dataset.

The idea ist to create a Multitarget tensor which has a number of 0/1 entries equal to the number of different objects in the images. For each images this tensor has a 1 for those objects included in the image.

Next steps: DataBlock, Dataset, Dataloader

Recap Datablock:<br>  
The data block API takes its name from the way it's designed: every bit needed to build the DataLoaders object (type of inputs, targets, how to label, split...) is encapsulated in a block, and you can mix and match those blocks. 

The Datablock API is there so we can tell the DataLoader how to get the input data and lalbels, how train and valid portions should be generated and whether we need to resize or do other transformations to the input. 

Possible structure:<br>
dblock = DataBlock(<br>blocks = <br>
                   get_items =<br>
                   get_x = <br>
                   get_y = <br>
                   splitter = <br>
                   item_tfms =<br>
                   batch_tfms = <br>)

Recap <br>Dataset:<br>
A collection that returns a tuple of your independent and dependent variable for a single item <br>
Datasets:
An object that contains a training Dataset and a validation Dataset

In the case of the Pascal data we need to specify the two blocks (ImageBlock and MulticategoryBlock), a get_x function with the filename, a get_y function extracting the label, a splitter which takes the last colum into account (is_valid) and a item_tfms in order to resize:

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y,
                   item_tfms = RandomResizedCrop(128, min_scale=0.35))
dls = dblock.dataloaders(df)

With this we could already build a learner:<br>
learn = cnn_leaner(dls, resnet18)<br>

We don't even need to chose a #loss function. FastAI will chose the appropriate fct itself. In the case of Multicategorical data it's called binary_cross_entropy, which is sigmoid, log and mean.<br>
Explicitly this is loss_func = nn.BCEWithLogitsLoss()<br>

But we do need to chose a #metric which is accuracy_multi.  

In [ ]:
def accuracy_multi(inp, targ, thresh=0.5, sigmoid=True):
    "Compute accuracy when `inp` and `targ` are the same size."
    if sigmoid: inp = inp.sigmoid()
    return ((inp>thresh)==targ.bool()).float().mean()

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.2))
learn.fine_tune(3, base_lr=3e-3, freeze_epochs=4)